In [1]:
import geopandas as gpd
import pandas as pd

#from geocube.api.core import make_geocube

In [ ]:
###############################################################################################################

In [2]:
import numpy as np
import math

In [3]:
test = gpd.read_file("/home/philipp/Data/GIS/GIS_grid/test_set/grid_test_set.shp")
tamsweg = gpd.read_file("/mnt/data_2/2018_tamsweg/vector/grid/512_02m/tamsweg_512_02m.shp")
mariazell = gpd.read_file("/mnt/data_2/2017_mariazell/vector/grid/512_02m/mariazell_512_02m.shp")
traun = gpd.read_file("/mnt/data_2/2017_traun/vector/grid/512_02m/traun_512_02m.shp")
pinzgau = gpd.read_file("/mnt/data_2/2018_pinzgau/vector/grid/512_02m/pinzgau_512_02m.shp")

test_indxs = np.array(test['id'])
tamsweg = np.array(tamsweg['id'])
mariazell = np.array(mariazell['id'])
traun = np.array(traun['id'])
pinzgau = np.array(pinzgau['id'])

In [4]:
test_indxs

array([ 48905,  48906,  48955, ..., 308607, 308608, 308645])

In [5]:
def reindex(train_idxs, test_idxs):
    sub_train = []
    sub_test = []
    for i in range(train_idxs.size):
        if train_idxs[i] in test_idxs:
            sub_test.append(i)
        else:
            sub_train.append(i)
    return {'train':np.array(sub_train), 'test':np.array(sub_test)}

In [6]:
def transform_indices(indices, interval, size, start=0):
    '''
    transform indices from id (shp) to index (hdf5)
    '''
    temp = []
    
    # loop for as long as interval is a multiple
    for i in range(0, math.floor(size/interval)):
        # filter for interval
        inteval_idxs = indices[(indices>=i*interval) & (indices<(i+1)*interval)]
        # creat helper variable
        j = i*3*interval
        # append all created sub indices
        temp.append(np.concatenate([inteval_idxs+j, inteval_idxs+j+interval, inteval_idxs+j+2*interval, inteval_idxs+j+3*interval]))
    
    # rest of interval
    rest = size % interval
    # creat helper variable
    j = (size - rest) * 3
    inteval_idxs = indices[(indices>=size-rest) & (indices<size)]
    # append all created sub indices
    temp.append(np.concatenate([inteval_idxs+j, inteval_idxs+j+rest, inteval_idxs+j+2*rest, inteval_idxs+j+3*rest]))

    # concatonate all
    new_idxs = np.concatenate(temp)
    
    return new_idxs + start

In [7]:
train_reg = []
test_reg = []
start = 0

for region in [tamsweg, mariazell, traun, pinzgau]:
    
    # reindex 
    idxs_ser = reindex(region, test_indxs)

    # trasform indices based on quatering the data in baches of 1000
    train_reg.append(transform_indices(idxs_ser['train'], 1000, len(region), start=start))
    test_reg.append(transform_indices(idxs_ser['test'], 1000, len(region), start=start))
    start += len(region)*4

train = np.concatenate(train_reg)
test = np.concatenate(test_reg)

In [8]:
print(train)
print(train.size)
print(test)
print(test.size)

[     0      1      2 ... 172821 172822 172823]
137608
[   769    770    771 ... 159593 159594 159595]
35216


In [84]:
np.save("/home/philipp/Data/edin_dataset/indices/numpy/train_all.npy", train)
np.save("/home/philipp/Data/edin_dataset/indices/numpy/test_all.npy", test)

In [14]:
for i in range(4):
    np.save("/mnt/data_2/dataset/indices/indices_test_256_df_{}.npy".format(i), test_reg[i])